In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# open text file and read in data as `text`
with open('Data/BloodyCromLyrics.txt', 'r') as f:
    text = f.read()
text = text.lower()

In [3]:
text[:100]

'the goblin creeper\nforged by elves\nis the sword of thorin\nis the tusk of rage\n\nthe sword of king und'

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([25, 27, 28, 10, 18, 12, 23, 17, 29, 24, 10,  2, 21, 28, 28, 30, 28,
       21, 31,  3, 12, 21, 18, 28,  6, 10, 23,  4, 10, 28, 17, 11, 28,  5,
       31, 29,  5, 10, 25, 27, 28, 10,  5, 13, 12, 21,  6, 10, 12,  3, 10,
       25, 27, 12, 21, 29, 24, 31, 29,  5, 10, 25, 27, 28, 10, 25, 16,  5,
        0, 10, 12,  3, 10, 21,  7, 18, 28, 31, 31, 25, 27, 28, 10,  5, 13,
       12, 21,  6, 10, 12,  3, 10,  0, 29, 24, 18, 10, 16, 24,  6])

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
#check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[25 27 28 10 18 12 23 17 29 24]
 [10 30  7  5 25 31 31 24 12 13]
 [12 21 18 12 25 27 10 27  7 14]
 [27 28 10 23  7 25 25 17 28 10]
 [ 7 11 28 10  2 16 25 10 25 27]
 [12  5 31  0 29 17 17 10 25 27]
 [21 12 25 31 21 28  7 17 14  5]
 [10 27  7 11 28 10 25 12 10 21]]

y
 [[27 28 10 18 12 23 17 29 24 10]
 [30  7  5 25 31 31 24 12 13 10]
 [21 18 12 25 27 10 27  7 14 14]
 [28 10 23  7 25 25 17 28 10 29]
 [11 28 10  2 16 25 10 25 27 28]
 [ 5 31  0 29 17 17 10 25 27 28]
 [12 25 31 21 28  7 17 14  5 10]
 [27  7 11 28 10 25 12 10 21 28]]


In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.05, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [55]:
# define and print the net
n_hidden = 150
n_layers = 3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(32, 150, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=150, out_features=32, bias=True)
)


In [56]:
batch_size = 5
seq_length = 30
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.1994... Val Loss: 3.1658
Epoch: 1/50... Step: 20... Loss: 3.0101... Val Loss: 3.0605
Epoch: 2/50... Step: 30... Loss: 3.0880... Val Loss: 3.0270
Epoch: 2/50... Step: 40... Loss: 3.0622... Val Loss: 3.0086
Epoch: 3/50... Step: 50... Loss: 2.9803... Val Loss: 3.0106
Epoch: 3/50... Step: 60... Loss: 3.0055... Val Loss: 3.0095
Epoch: 3/50... Step: 70... Loss: 2.9772... Val Loss: 3.0051
Epoch: 4/50... Step: 80... Loss: 2.9784... Val Loss: 3.0072
Epoch: 4/50... Step: 90... Loss: 2.9781... Val Loss: 3.0046
Epoch: 5/50... Step: 100... Loss: 2.9756... Val Loss: 3.0071
Epoch: 5/50... Step: 110... Loss: 2.8974... Val Loss: 3.0046
Epoch: 5/50... Step: 120... Loss: 2.9914... Val Loss: 3.0049
Epoch: 6/50... Step: 130... Loss: 2.9701... Val Loss: 3.0075
Epoch: 6/50... Step: 140... Loss: 2.9113... Val Loss: 3.0036
Epoch: 7/50... Step: 150... Loss: 2.9971... Val Loss: 3.0049
Epoch: 7/50... Step: 160... Loss: 3.0480... Val Loss: 3.0020
Epoch: 8/50... Step: 170... Loss:

In [57]:
print(sample(net, 500, prime='sword', top_k=3))

sword dad yil the bragte the betin
the sword of toreos
the word the bestes in the charcin

the swoll doms
the cherte or the brock the blow is the swild the dinthen

the cizing the swells i feomh the colge
the browth of the brocg of the bridg of the brothers in the beat of the brinks of damin de the cheag
of mas and you hale the sling throus the swerd of doot
the corin the chaon
nungeon
dungeon

i bring you sall of morgen
the bobgin of the worrd for angeor the cores
the wirl you hale the sworg of me th


In [58]:
# change the name, for saving multiple files
model_name = 'lyricsgenerator.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [18]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [19]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [20]:
print(sample(net, 500, prime='dead', top_k=3))

dead
broodin mind you hate yourself
iâ´ll make you cut everyone
stormbringer master of the orcs

wild in the brackes
is the sword cuts heads
we can continue the quest

dungeon
dungeon
dungeon
dungeon
dungeon

we are in a grimmy room
full of bat he chaos
now you cer everyone
stormbringer mascer of the arfss trough the ages

dagger of morgul
on your own chess
fell the inon of dorin
che pork of this empty place
they found it in the darkness, the durinâ´s bane
a demon of the abyss from the morgoth hand




In [21]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [22]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="warlock"))

warlock bracksss from you'll every friend
iâ´m a pirt of a demon race
my brother is the mournblade

to be wielded by the king
the black sword from the chaos
now you cer everyofe
inothin the kead

we hand cove for unverd your elom

i lati nute durgaon
i wis your blood
ine the corrod champer
of the orcs

riding my horse i search the blood
brandish my sword
my holy sword
fighting in battle again and again
seaching the glory in death

who are you?
i'm the knight of doom

i drink your dlook
i drink you death
i bring you death
i bring you daith
i bring you pain
i wile ride over you head

in a werright of door

i drink sours from you'll every frinnds eypraog
the would in i grinty on mordot
creof now anf ala hear
cut che fris and the troll they wond through trees
footprint on che sword, the qousd
a fris mine you hate yourself
iâ´ll make you hete you sealm not pass

this is a foe behind any of you
the wizard skoks the fire eyes
uduns flame brights so bad
you shall not pass

this is a foe behind